In [63]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from scipy import optimize, integrate
import time
import os

In [64]:
mRb = 87*1.67e-27;
gRb = 2*np.pi*6.066e6;
mCs = 133*1.67e-27;
gCs = 2*np.pi*5.234e6;
k = 1.38e-23;
hb = 1.0545e-34;
fr = [74.434e3/2., 70.919e3/2.]
frerr = [368/2., 222/2.]
fa = [7.477e3/2, 7.135e3/2]
faerr = [56/2., 31/2.]

def Tdoppler(m, g):
    return hb*g/(2*k)

In [65]:
print('Rb: {:.0f} uK'.format(Tdoppler(mRb, gRb)*1e6))
print('Cs: {:.0f} uK'.format(Tdoppler(mCs, gCs)*1e6))

Rb: 146 uK
Cs: 126 uK


In [66]:
def a_stdev(m, T, f):
    # m: mass, T: temp, w: beam waist
    return np.sqrt(k*T/(m*np.power(np.array(2*np.pi*f), 2)))

def a_stdev_err(m, T, f, ferr):
    # m: mass, T: temp, w: beam waist
    return 2*np.pi*ferr*np.sqrt(k*T/(m*np.power(np.array(2*np.pi*f), 4)))

# checking some known numbers
print('Rb: {:.3f}, {:.3f}, {:.3f}  um'.format(*(1e6*a_stdev(mRb, Tdoppler(mRb, gRb), np.array([4e4, 4e4, 3.2e3])))))
print('Rb: {:.3f}, {:.3f}, {:.3f}  um'.format(*(1e6*a_stdev(mRb, Tdoppler(mRb, gRb), np.array([4.8e4, 4.8e4, 4.7e3])))))
print('Cs: {:.3f}, {:.3f}, {:.3f}  um'.format(*(1e6*a_stdev(mCs, Tdoppler(mCs, gCs), np.array([5.3e4, 5.3e4, 5e3])))))
# most recent numbers
u_power = np.array([84.2, 135.6, 105.625, 72])
u_fac = np.array([84.2/90., 135.6/90., 105.625/90., 72./90.])  # power ratio from nominal, trap power ratio
acloudr = np.array([
    a_stdev(mCs, 1, fr[0]*np.sqrt(u_fac)),
    a_stdev(mRb, 1, fr[1]*np.sqrt(u_fac))
])
acloudr_err = np.array([
    a_stdev_err(mCs, 1, fr[0]*np.sqrt(u_fac), frerr[0]*np.sqrt(u_fac)),
    a_stdev_err(mRb, 1, fr[1]*np.sqrt(u_fac), frerr[1]*np.sqrt(u_fac))
])
aclouda = np.array([
    a_stdev(mCs, 1, fa[0]*np.sqrt(u_fac)),
    a_stdev(mRb, 1, fa[1]*np.sqrt(u_fac))
])
aclouda_err = np.array([
    a_stdev_err(mCs, 1, fa[0]*np.sqrt(u_fac), faerr[0]*np.sqrt(u_fac)),
    a_stdev_err(mRb, 1, fa[1]*np.sqrt(u_fac), faerr[1]*np.sqrt(u_fac))
])

print('\n    fr (um/rtK),   fa (um/rtK)')
for i in range(len(u_fac)):
    print('Cs: {:.2f}({:.2f}),   {:.2f}({:.2f})'.format(1e6*acloudr[0,i], 1e6*acloudr_err[0,i], 1e6*aclouda[0,i], 1e6*aclouda_err[0,i]))
    print('Rb: {:.2f}({:.2f}),   {:.2f}({:.2f})'.format(1e6*acloudr[1,i], 1e6*acloudr_err[1,i], 1e6*aclouda[1,i], 1e6*aclouda_err[1,i]))
    print('-'*15)

Rb: 0.468, 0.468, 5.849  um
Rb: 0.390, 0.390, 3.982  um
Cs: 0.265, 0.265, 2.812  um

    fr (um/rtK),   fa (um/rtK)
Cs: 34.85(0.17),   346.93(2.60)
Rb: 45.22(0.14),   449.51(1.95)
---------------
Cs: 27.46(0.14),   273.38(2.05)
Rb: 35.64(0.11),   354.22(1.54)
---------------
Cs: 31.12(0.15),   309.75(2.32)
Rb: 40.38(0.13),   401.34(1.74)
---------------
Cs: 37.69(0.19),   375.17(2.81)
Rb: 48.91(0.15),   486.11(2.11)
---------------


In [67]:
acloudr

array([[3.48497338e-05, 2.74615892e-05, 3.11151680e-05, 3.76868000e-05],
       [4.52245667e-05, 3.56369572e-05, 4.03782134e-05, 4.89062329e-05]])

In [68]:
def overlap_density(sx, sa):
    # sx -> sigma_x, sa -> sigma_z
    return 1./(2*np.pi*np.sum(np.power(sx, 2)))/np.sqrt(2*np.pi*np.sum(np.power(sa, 2)))

def overlap_density_err(sx, sx_err, sa, sa_err):
    # sx -> sigma_x, sa -> sigma_z
    return np.sqrt(
        np.sum(np.power(sx*sx_err/(np.sqrt(2)*np.power(np.pi, 1.5)*np.power(np.sum(np.power(sx, 2)), 2)*np.sqrt(np.sum(np.power(sa, 2)))), 2)) +
        np.sum(np.power(sa*sa_err/(np.power(2*np.pi, 1.5)*np.sum(np.power(sx, 2))*np.sqrt(np.sum(np.power(sa, 2)))), 1.5))
    )

In [69]:
def vbar(m1, T1, m2, T2):
    return np.sqrt(8*k*(T1/m1 + T2/m2)/np.pi)

def vbar_csrb(Tc, Tr):
    return vbar(mCs, Tc, mRb, Tr)

print vbar_csrb(0.0001, 0.00018)

0.2436357376544271


## 42 (2018_07_09_10_18)

'2018_07_09_10_18_02_rbcs-collision-42-unpol-4_5ms'
'2018_07_09_11_08_47_rbcs-collision-42-unpol-10ms'
'2018_07_09_11_33_51_rbcs-collision-42-unpol-30ms'
'2018_07_09_11_44_41_rbcs-collision-42-unpol-60ms'
'2018_07_09_12_03_06_rbcs-collision-42-unpol-100ms'
'2018_07_09_13_15_36_rbcs-collision-42-unpol-300ms'
'2018_07_09_14_25_53_rbcs-collision-42-unpol-600ms'

In [70]:
b42 = [0.49899089, 0.5545054, 0.43417923, 0.42319934, 0.36130645]
b42err = [0.05590329, 0.0617219, 0.03776802, 0.05243504, 0.02613373]

In [71]:
# b42 = [1.02148513, 1.08219925e+00, 0.51357683, 1.16771006, 0.56879905]
# b42err = [0.18750717, 0.21589205, 0.10867261, 0.22955, 0.09234445]

'2018_07_08_22_21_54_temperature'
make sure to take temperature measurements closer in the future

In [72]:
# upgrade the temperature fitting so that I can extract an error
t42 = np.array([  # uK  # old
    [84.5, 104., 84.8, 82, 113.1],  # cs
    [79.1, 80.8, 62.2, 69.1, 68.8]  # rb
])
# t42 = np.array([  # uK
#     [86.2, 104.6, 81.9, 82, 113.1],  # cs
#     [78.9, 79.8, 69.4, 69.1, 68.8]  # rb
# ])
# t42err = np.array([  # uK
#     [3.60, 5.33, 5.45, , ],  # cs
#     [10.2, 13.8, 9.09, , ]  # rb
# ])

# I took an extra data point at high density 
acloudr42 = np.copy(acloudr)
acloudr42 = np.insert(acloudr42, 3, acloudr42[:,2], axis=1)
u_power42 = np.copy(u_power)
u_power42 = np.insert(u_power42, 3, u_power42[2], axis=0)
# acloudr42 = np.insert(acloudr42, 4, acloudr42[:,-1], axis=1)
print(acloudr)
print('+'*6)
print(acloudr42)
print('='*10)
print(u_power)
print('+'*6)
print(u_power42)

acloudr42_err = np.copy(acloudr_err)
acloudr42_err = np.insert(acloudr42_err, 3, acloudr42_err[:,2], axis=1)
# acloudr42_err = np.insert(acloudr42_err, 4, acloudr42_err[:,-1], axis=1)

aclouda42 = np.copy(aclouda)
aclouda42 = np.insert(aclouda42, 3, aclouda42[:,2], axis=1)
# aclouda42 = np.insert(aclouda42, 4, aclouda42[:,-1], axis=1)
aclouda42_err = np.copy(aclouda_err)
aclouda42_err = np.insert(aclouda42_err, 3, aclouda42_err[:,2], axis=1)
# aclouda42_err = np.insert(aclouda42_err, 4, aclouda42_err[:,-1], axis=1)

sigma42r = acloudr42[:,:len(t42[0])]*np.sqrt(1e-6*t42)
sigma42r_err = acloudr42_err[:,:len(t42[0])]*np.sqrt(1e-6*t42)

sigma42a = aclouda42[:,:len(t42[0])]*np.sqrt(1e-6*t42)
sigma42a_err = aclouda42_err[:,:len(t42[0])]*np.sqrt(1e-6*t42)

[[3.48497338e-05 2.74615892e-05 3.11151680e-05 3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 4.03782134e-05 4.89062329e-05]]
++++++
[[3.48497338e-05 2.74615892e-05 3.11151680e-05 3.11151680e-05
  3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 4.03782134e-05 4.03782134e-05
  4.89062329e-05]]
[ 84.2   135.6   105.625  72.   ]
++++++
[ 84.2   135.6   105.625 105.625  72.   ]


In [73]:
print(sigma42r*1e6)
print(sigma42r_err*1e6)

print(sigma42a*1e6)
print(sigma42a_err*1e6)

[[0.32035228 0.28005436 0.28652998 0.28175983 0.4007934 ]
 [0.40221907 0.3203364  0.31845076 0.33564959 0.40565649]]
[[0.00158381 0.00138458 0.0014166  0.00139301 0.00198151]
 [0.00125908 0.00100276 0.00099686 0.00105069 0.00126984]]
[[3.18912687 2.78795856 2.85242381 2.80493663 3.98992326]
 [3.99789406 3.18401365 3.16527113 3.33622052 4.03206066]]
[[0.0238854  0.02088079 0.02136361 0.02100795 0.02988307]
 [0.01736997 0.01383384 0.0137524  0.01449514 0.01751841]]


In [74]:
od42 = [1e-16*overlap_density(sigma42r[:,i], sigma42a[:,i]) for i in range(len(sigma42r[0]))]  # 10^-10 cm^-3
od42_err = [1e-16*overlap_density_err(sigma42r[:,i], sigma42r_err[:,i], sigma42a[:,i], sigma42a_err[:,i]) for i in range(len(sigma42r[0]))]
print(od42)
print(od42_err)

[4.695618648047807, 8.286781081074869, 8.120231942640398, 7.585133340173361, 3.4420473460962318]
[0.02546192128772027, 0.046094557859594244, 0.04560290599088632, 0.041680703465827584, 0.020038999866821578]


## 31 (2018_07_09_10_18)

'2018_07_09_22_50_18_rbcs-collision-31-unpol'
'2018_07_09_23_11_33_rbcs-collision-31-unpol-pt2'

In [75]:
# b31 = [0.03101263, 0.05244247]
# b31err = [0.01624713, 0.02853062]
b31 = [0.01931074, 4.22298583e-16]
b31err = [0.01723351, 0.03178765]

'2018_07_08_22_21_54_temperature'
make sure to take temperature measurements closer in the future

In [76]:
# upgrade the temperature fitting so that I can extract an error
t31 = np.array([  # uK
    [84.5, 92.5],  # cs
    [79.1, 94.1]  # rb
])
sigma31r = acloudr[:,:len(b31)]*np.sqrt(1e-6*t31[:,:len(b31)])
sigma31r_err = acloudr_err[:,:len(b31)]*np.sqrt(1e-6*t31[:,:len(b31)])

u_power31 = np.copy(u_power)[:len(b31)]

sigma31a = aclouda[:,:len(b31)]*np.sqrt(1e-6*t31[:,:len(b31)])
sigma31a_err = aclouda_err[:,:len(b31)]*np.sqrt(1e-6*t31[:,:len(b31)])

In [77]:
print(sigma31r*1e6)
print(sigma31r_err*1e6)

print(sigma31a*1e6)
print(sigma31a_err*1e6)

[[0.32035228 0.26411711]
 [0.40221907 0.34569685]]
[[0.00158381 0.00130579]
 [0.00125908 0.00108215]]
[[3.18912687 2.62930223]
 [3.99789406 3.43608622]]
[[0.0238854  0.01969251]
 [0.01736997 0.01492904]]


In [78]:
od31 = [1e-16*overlap_density(sigma31r[:,i], sigma31a[:,i]) for i in range(len(b31))]  # 10^-10 cm^-3
od31_err = [1e-16*overlap_density_err(sigma31r[:,i], sigma31r_err[:,i], sigma31a[:,i], sigma31a_err[:,i]) for i in range(len(b31))]
print(od31)
print(od31_err)

[4.695618648047807, 7.753714989637125]
[0.02546192128772027, 0.041689703930981925]


## 32 (2018_07_10)

'2018_07_10_16_32_34_rbcs-collisions-32-unpol-pt1'
'2018_07_10_17_20_51_rbcs-collisions-32-unpol-pt2'

In [79]:
# b32 = [0.30499042,4.52109619e-01]
# b32err = [0.04149042, 0.04853909]
b32 = [0.31543102,4.31680344e-01,0.28897518]
b32err = [0.04331461, 0.05378444,0.01913829]

'2018_07_10_18_39_24_rbcs-collisions-32-unpol-temp'

In [80]:
# upgrade the temperature fitting so that I can extract an error
t32 = np.array([  # uK
    [97.4, 99.2, 83.3],  # cs
    [75.3, 89.5, 67.2]  # rb
])

# I took an extra data point at high density 
acloudr32 = np.copy(acloudr)
acloudr32 = np.insert(acloudr32, 2, acloudr42[:,-1], axis=1)
# acloudr42 = np.insert(acloudr42, 4, acloudr42[:,-1], axis=1)
print(acloudr)
print(acloudr32)
acloudr32_err = np.copy(acloudr_err)
acloudr32_err = np.insert(acloudr32_err, 2, acloudr32_err[:,-1], axis=1)
# acloudr42_err = np.insert(acloudr42_err, 4, acloudr42_err[:,-1], axis=1)
u_power32 = np.copy(u_power)
u_power32 = np.insert(u_power32, 2, u_power32[-1], axis=0)

aclouda32 = np.copy(aclouda)
aclouda32 = np.insert(aclouda32, 2, aclouda32[:,-1], axis=1)
# aclouda42 = np.insert(aclouda42, 4, aclouda42[:,-1], axis=1)
aclouda32_err = np.copy(aclouda_err)
aclouda32_err = np.insert(aclouda32_err, 3, aclouda32_err[:,2], axis=1)
# aclouda42_err = np.insert(aclouda42_err, 4, aclouda42_err[:,-1], axis=1)

sigma32r = acloudr32[:,:len(b32)]*np.sqrt(1e-6*t32[:,:len(b32)])
sigma32r_err = acloudr32_err[:,:len(b32)]*np.sqrt(1e-6*t32[:,:len(b32)])

sigma32a = aclouda32[:,:len(b32)]*np.sqrt(1e-6*t32[:,:len(b32)])
sigma32a_err = aclouda32_err[:,:len(b32)]*np.sqrt(1e-6*t32[:,:len(b32)])

[[3.48497338e-05 2.74615892e-05 3.11151680e-05 3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 4.03782134e-05 4.89062329e-05]]
[[3.48497338e-05 2.74615892e-05 3.76868000e-05 3.11151680e-05
  3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 4.89062329e-05 4.03782134e-05
  4.89062329e-05]]


In [81]:
print(sigma32r*1e6)
print(sigma32r_err*1e6)

print(sigma32a*1e6)
print(sigma32a_err*1e6)

[[0.34393704 0.27351522 0.34396303]
 [0.39243877 0.33714144 0.40091181]]
[[0.00170042 0.00135225 0.00170055]
 [0.00122846 0.00105536 0.00125499]]
[[3.42391458 2.72286105 3.42417334]
 [3.90068184 3.35104885 3.98490043]]
[[0.02564387 0.02039324 0.02117382]
 [0.0169476  0.01455957 0.01429447]]


In [82]:
od32 = [1e-16*overlap_density(sigma32r[:,i], sigma32a[:,i]) for i in range(len(b32))]  # 10^-10 cm^-3
od32_err = [1e-16*overlap_density_err(sigma32r[:,i], sigma32r_err[:,i], sigma32a[:,i], sigma32a_err[:,i]) for i in range(len(b32))]
print(od32)
print(od32_err)

[4.492565151011152, 7.802124673306368, 4.330851671026311]
[0.02500931316886158, 0.04249073670317238, 0.023949603510933136]


## 41 (2018_07_10)

'2018_07_10_11_39_53_rbcs-collisions-41-unpol-pt1'
'2018_07_10_11_57_03_rbcs-collisions-41-unpol-pt2'

In [83]:
#b41 = [0.53319096, 0.64654081, 0.81842229]  # old model
#b41err = [0.06883381, 0.07750669, 0.06680952]
b41 = [0.56903, 0.79787923, 0.81842229, 0.5090469]  # new model
b41err = [0.07386109, 0.08785919, 0.06680952, 0.04280821]

'2018_07_10_15_45_16_rbcs-collisions-41-temp'

In [84]:
# upgrade the temperature fitting so that I can extract an error
t41 = np.array([  # uK
    [91.1, 95.4, 95.6, 74],  # cs
    [83.2, 91.9, 91.3, 67.2]  # rb
])

# I took an extra data point at high density 
acloudr41 = np.copy(acloudr)
acloudr41 = np.insert(acloudr41, 2, acloudr41[:,1], axis=1)
acloudr41 = np.insert(acloudr41, 3, acloudr41[:,-1], axis=1)
print(acloudr)
print(acloudr41)
acloudr41_err = np.copy(acloudr_err)
acloudr41_err = np.insert(acloudr41_err, 2, acloudr41_err[:,1], axis=1)
acloudr41_err = np.insert(acloudr41_err, 3, acloudr41_err[:,-1], axis=1)

u_power41 = np.copy(u_power)
u_power41 = np.insert(u_power41, 2, u_power41[1], axis=0)
u_power41 = np.insert(u_power41, 3, u_power41[-1], axis=0)

aclouda41 = np.copy(aclouda)
aclouda41 = np.insert(aclouda41, 2, aclouda41[:,1], axis=1)
aclouda41 = np.insert(aclouda41, 3, aclouda41[:,-1], axis=1)
aclouda41_err = np.copy(aclouda_err)
aclouda41_err = np.insert(aclouda41_err, 2, aclouda41_err[:,1], axis=1)
aclouda41_err = np.insert(aclouda41_err, 3, aclouda41_err[:,-1], axis=1)

sigma41r = acloudr41[:,:len(b41)]*np.sqrt(1e-6*t41[:,:len(b41)])
sigma41r_err = acloudr41_err[:,:len(b41)]*np.sqrt(1e-6*t41[:,:len(b41)])

sigma41a = aclouda41[:,:len(b41)]*np.sqrt(1e-6*t41[:,:len(b41)])
sigma41a_err = aclouda41_err[:,:len(b41)]*np.sqrt(1e-6*t41[:,:len(b41)])

[[3.48497338e-05 2.74615892e-05 3.11151680e-05 3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 4.03782134e-05 4.89062329e-05]]
[[3.48497338e-05 2.74615892e-05 2.74615892e-05 3.76868000e-05
  3.11151680e-05 3.76868000e-05]
 [4.52245667e-05 3.56369572e-05 3.56369572e-05 4.89062329e-05
  4.03782134e-05 4.89062329e-05]]


In [85]:
print(sigma41r*1e6)
print(sigma41r_err*1e6)

print(sigma41a*1e6)
print(sigma41a_err*1e6)

[[0.33262788 0.26822537 0.26850638 0.32419411]
 [0.41251152 0.34163186 0.34051481 0.40091181]]
[[0.0016445  0.0013261  0.00132749 0.00160281]
 [0.0012913  0.00106942 0.00106592 0.00125499]]
[[3.31133128 2.67020024 2.67299773 3.22737254]
 [4.10019681 3.39568188 3.3845788  3.98490043]]
[[0.02480066 0.01999882 0.02001978 0.02417184]
 [0.01781445 0.01475349 0.01470525 0.01731351]]


In [86]:
od41 = [1e-16*overlap_density(sigma41r[:,i], sigma41a[:,i]) for i in range(len(b41))]  # 10^-10 cm^-3
od41_err = [1e-16*overlap_density_err(sigma41r[:,i], sigma41r_err[:,i], sigma41a[:,i], sigma41a_err[:,i]) for i in range(len(b41))]
print(od41)
print(od41_err)

[4.290255816819904, 7.791010128799664, 7.828998422704749, 4.657825057089577]
[0.023330542653599483, 0.04211481797188531, 0.042359108956692086, 0.025346670296528675]


## 4020 (2018_08_06)

In [87]:
b4020 = [0.49842794]
b4020err = [0.0472946]

'2018_07_08_22_21_54_temperature'
make sure to take temperature measurements closer in the future

In [88]:
# upgrade the temperature fitting so that I can extract an error
# t42 = np.array([  # uK  # old
#     [84.5, 104., 84.8, 82, 113.1],  # cs
#     [79.1, 80.8, 62.2, 69.1, 68.8]  # rb
# ])
t4020 = np.array([  # uK
    [82.5],  # cs
    [68.7]  # rb
])
t4020err = np.array([  # uK
    [11.5],  # cs
    [9.69]  # rb
])

# I took an extra data point at high density 
acloudr4020 = np.copy(acloudr)
# acloudr4020 = np.insert(acloudr42, 3, acloudr42[:,2], axis=1)
# acloudr42 = np.insert(acloudr42, 4, acloudr42[:,-1], axis=1)
# print(acloudr)
# print(acloudr42)
acloudr4020_err = np.copy(acloudr_err)
# acloudr4020_err = np.insert(acloudr42_err, 3, acloudr42_err[:,2], axis=1)
# acloudr42_err = np.insert(acloudr42_err, 4, acloudr42_err[:,-1], axis=1)

aclouda4020 = np.copy(aclouda)
# aclouda42 = np.insert(aclouda42, 3, aclouda42[:,2], axis=1)
# aclouda42 = np.insert(aclouda42, 4, aclouda42[:,-1], axis=1)
aclouda4020_err = np.copy(aclouda_err)
# aclouda42_err = np.insert(aclouda42_err, 3, aclouda42_err[:,2], axis=1)
# aclouda42_err = np.insert(aclouda42_err, 4, aclouda42_err[:,-1], axis=1)

sigma4020r = acloudr4020[:,:len(t4020[0])]*np.sqrt(1e-6*t4020)
sigma4020r_err = acloudr4020_err[:,:len(t4020[0])]*np.sqrt(1e-6*t4020)

sigma4020a = aclouda4020[:,:len(t4020[0])]*np.sqrt(1e-6*t4020)
sigma4020a_err = aclouda4020_err[:,:len(t4020[0])]*np.sqrt(1e-6*t4020)

In [89]:
print(sigma4020r*1e6)
print(sigma4020r_err*1e6)

print(sigma4020a*1e6)
print(sigma4020a_err*1e6)

[[0.31653843]
 [0.37484592]]
[[0.00156496]
 [0.00117339]]
[[3.15115972]
 [3.72581605]]
[[0.02360104]
 [0.01618785]]


In [90]:
od4020 = [1e-16*overlap_density(sigma4020r[:,i], sigma4020a[:,i]) for i in range(len(sigma4020r[0]))]  # 10^-10 cm^-3
od4020_err = [1e-16*overlap_density_err(sigma4020r[:,i], sigma4020r_err[:,i], sigma4020a[:,i], sigma4020a_err[:,i]) for i in range(len(sigma4020r[0]))]
print(od4020)
print(od4020_err)

[5.405672397566167]
[0.029754417278596994]


# Results

In [91]:
fig, ax = plt.subplots(figsize=(2*3.375,4.5))

marker_styles = [
    dict(color='cornflowerblue', marker='^', markeredgecolor='b', linestyle='None', markersize=8),
    dict(color='indianred', marker='o', markeredgecolor='r', linestyle='None', markersize=8),
    dict(color='chartreuse', marker='d', markeredgecolor='g', linestyle='None', markersize=8),
    dict(color='plum', marker='v', markeredgecolor='darkmagenta', linestyle='None', markersize=8),
    dict(color='gainsboro', marker='s', markeredgecolor='dimgray', linestyle='None', markersize=8),
]

ods = [od42, od41, od32, od31, od4020]
od_errs = [od42_err, od41_err, od32_err, od31_err, od4020_err]
bs = [b42, b41, b32, b31, b4020]
b_errs = np.array([b42err, b41err, b32err, b31err, b4020err])

leg = ['42','41','32','31','4020']
as_ = []
xlim = 10
for hf in range(len(ods)):
    W = np.diag(np.power(b_errs[hf], -0.5))  # weight matrix
    Aw = np.dot(W, np.array(ods[hf])[:,np.newaxis])
    Bw = np.dot(bs[hf], W)
    a, _, _, _ = np.linalg.lstsq(Aw, Bw)
    print('{}: {:.4f} x10^-12 cm^3/s'.format(leg[hf], a[0]*1e2))
    as_.append(a[0])
    xs = np.linspace(0,xlim,2)
    ax.plot(xs, a*xs, color=marker_styles[hf]['markeredgecolor'], linestyle='--')
    ax.errorbar(
        ods[hf], bs[hf], xerr=od_errs[hf], yerr=b_errs[hf],
        label=leg[hf], **marker_styles[hf]
    )
    
ax.set_xlim(0,xlim)
ax.set_ylim(0,1.)
ax.grid()
ax.set_xlabel(r"Density Overlap ($\times 10^{10}\cdot$cm$^{-3}$)", fontsize=14)
ax.set_ylabel(r"$\gamma_{CsRb}$ (s$^{-1}$)", fontsize=14)
ax.legend(bbox_to_anchor=(0.2, 0.95), loc=1, borderaxespad=0., fontsize=14)
plt.tight_layout()

out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/'
fmts = ['pdf', 'png', 'eps']
fn = os.path.join(out, 'beta-data_{}.{}')
for fmt in fmts:
    fig.savefig(fn.format(time.strftime("%Y%m%d_%H%M%S"), fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


42: 6.6310 x10^-12 cm^3/s
41: 10.7896 x10^-12 cm^3/s
32: 6.2352 x10^-12 cm^3/s
31: 0.1659 x10^-12 cm^3/s
4020: 9.2205 x10^-12 cm^3/s


In [92]:
fig, ax = plt.subplots(figsize=(2*3.375,4.5))

u_pows = [u_power42, u_power41, u_power32, u_power31]

leg = ['42','41','32','31']


for hf in range(len(u_pows)):
#     W = np.diag(np.power(b_errs[hf], -0.5))  # weight matrix
#     Aw = np.dot(W, np.array(ods[hf])[:,np.newaxis])
#     Bw = np.dot(bs[hf], W)
#     a, _, _, _ = np.linalg.lstsq(Aw, Bw)
#     print('{}: {:.4f} x10^-12 cm^3/s'.format(leg[hf], a[0]*1e2))
#     as_.append(a[0])
#     xs = np.linspace(0,xlim,2)
#     ax.plot(xs, a*xs, color=marker_styles[hf]['markeredgecolor'], linestyle='--')
    print(hf)
    print(u_pows[hf])
    print(bs[hf])
    ax.errorbar(
        u_pows[hf][:len(bs[hf])], bs[hf], yerr=b_errs[hf],
        label=leg[hf], **marker_styles[hf]
    )
ax.set_xlim(0,140)
ax.set_ylim(0,1.)
ax.grid()
ax.set_xlabel(r"ODT Power (mV)", fontsize=14)
ax.set_ylabel(r"$\gamma_{CsRb}$ (s$^{-1}$)", fontsize=14)
ax.legend(bbox_to_anchor=(0.2, 0.95), loc=1, borderaxespad=0., fontsize=14)
plt.tight_layout()


FigureCanvasNbAgg()

0
[ 84.2   135.6   105.625 105.625  72.   ]
[0.49899089, 0.5545054, 0.43417923, 0.42319934, 0.36130645]
1
[ 84.2   135.6   135.6    72.    105.625  72.   ]
[0.56903, 0.79787923, 0.81842229, 0.5090469]
2
[ 84.2   135.6    72.    105.625  72.   ]
[0.31543102, 0.431680344, 0.28897518]
3
[ 84.2 135.6]
[0.01931074, 4.22298583e-16]


In [93]:
xsec_cm2 = np.divide(np.array(as_)*1e-10, 100*np.array([
    vbar_csrb(*(t42[:,0]*1e-6)),
    vbar_csrb(*(t41[:,0]*1e-6)),
    vbar_csrb(*(t32[:,0]*1e-6)),
    vbar_csrb(*(t31[:,0]*1e-6))
]))

ValueError: operands could not be broadcast together with shapes (5,) (4,) 

In [94]:
print(xsec_cm2)

NameError: name 'xsec_cm2' is not defined

In [95]:
b32

[0.31543102, 0.431680344, 0.28897518]

In [96]:
b41err

[0.07386109, 0.08785919, 0.06680952, 0.04280821]

In [97]:
print(t42[0])
print(t41[0])
print(t32[0])
print(t31[0])

[ 84.5 104.   84.8  82.  113.1]
[91.1 95.4 95.6 74. ]
[97.4 99.2 83.3]
[84.5 92.5]


In [98]:
print(t42[1])
print(t41[1])
print(t32[1])
print(t31[1])

[79.1 80.8 62.2 69.1 68.8]
[83.2 91.9 91.3 67.2]
[75.3 89.5 67.2]
[79.1 94.1]


In [99]:
print(t42[0]-t42[1])
print(t41[0]-t41[1])
print(t32[0]-t32[1])
print(t31[0]-t31[1])

[ 5.4 23.2 22.6 12.9 44.3]
[7.9 3.5 4.3 6.8]
[22.1  9.7 16.1]
[ 5.4 -1.6]


In [100]:
np.divide(b42,np.array(od42)*1e10)

array([1.06267337e-11, 6.69144502e-12, 5.34688212e-12, 5.57932631e-12,
       1.04968472e-11])

In [101]:
t42.T

array([[ 84.5,  79.1],
       [104. ,  80.8],
       [ 84.8,  62.2],
       [ 82. ,  69.1],
       [113.1,  68.8]])

In [102]:
def vbar(t12):
    return np.sqrt(8*k*(t12[0]/mCs + t12[1]/mRb)*1e-6/np.pi)

def therm(t12):
    return (mCs*t12[1] + mRb*t12[0])/(mCs+mRb)

In [103]:
t42_rel = 1e6*(0.5*(mRb*mCs/(mRb+mCs))*vbar(t42)**2)/k

In [104]:
fig, ax = plt.subplots(figsize=(2*3.375,4.5))
# ods = [od42, od41, od32, od31]
# od_errs = [od42_err, od41_err, od32_err, od31_err]
# bs = [b42, b41, b32, b31]
# b_errs = np.array([b42err, b41err, b32err, b31err])
ts = [t42, t41, t32, t31, t4020]

infiles = [ os.path.join('theory','20180801',fn) for fn in ['out.24.thermal','out.14.thermal','out.23.thermal'] ]

leg = ['42','41','32','31','4020']
fudge = 1
for hf in range(len(ts)):
    ax.errorbar(
        therm(ts[hf]),
        np.divide(bs[hf], ods[hf])*1e2,
        yerr=np.divide(b_errs[hf], ods[hf])*1e2,
        label=leg[hf], **marker_styles[hf]
    )
    if hf < len(infiles):
        thdata = np.loadtxt(infiles[hf])
        ax.plot(thdata[:,0], fudge*thdata[:,1]*1e12, color=marker_styles[hf]['color'])
    
ax.set_xlim(0,150)
ax.set_ylim(0,15)
ax.grid()
ax.set_xlabel(r"$T_{eff}$ ($\mu$K)", fontsize=14)
ax.set_ylabel(r"$\beta_{CsRb}$ ($\times 10^{-12}$ cm$^3$/s)", fontsize=14)
ax.legend(bbox_to_anchor=(0.2, 0.95), loc=1, borderaxespad=0., fontsize=14)
plt.tight_layout()

out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/'
fmts = ['pdf', 'png', 'eps']
fn = os.path.join(out, 'T-v-beta_{}.{}')
for fmt in fmts:
    fig.savefig(fn.format(time.strftime("%Y%m%d_%H%M%S"), fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

In [100]:
range(len(infiles))

[0, 1, 2]

array([[  2.50000000e+01,   1.49064000e-12],
       [  3.00000000e+01,   1.46060000e-12],
       [  3.50000000e+01,   1.43857000e-12],
       [  4.00000000e+01,   1.42059000e-12],
       [  4.50000000e+01,   1.40465000e-12],
       [  5.00000000e+01,   1.38972000e-12],
       [  5.50000000e+01,   1.37531000e-12],
       [  6.00000000e+01,   1.36119000e-12],
       [  6.50000000e+01,   1.34727000e-12],
       [  7.00000000e+01,   1.33354000e-12],
       [  7.50000000e+01,   1.32001000e-12],
       [  8.00000000e+01,   1.30670000e-12],
       [  8.50000000e+01,   1.29365000e-12],
       [  9.00000000e+01,   1.28088000e-12],
       [  9.50000000e+01,   1.26841000e-12],
       [  1.00000000e+02,   1.25626000e-12],
       [  1.05000000e+02,   1.24445000e-12],
       [  1.10000000e+02,   1.23298000e-12],
       [  1.15000000e+02,   1.22185000e-12],
       [  1.20000000e+02,   1.21107000e-12],
       [  1.25000000e+02,   1.20064000e-12],
       [  1.30000000e+02,   1.19055000e-12],
       [  